# Python Project: Tesla and Gamestop Stock Analysis Dashboard

Tesla and GameStop Stock Analysis Dashboard Project
- Part 1: Use yfinance to Extract Stock Data
- Part 2: Use Webscraping to Extract Tesla Revenue Data
- Part 3: Use yfinance to Extract Stock Data
- Part 4: Use Webscraping to Extract GME Revenue Data
- Part 5: Plot Tesla Stock Graph
- Part 6: Plot GameStop Stock Graph

Just a note: For plotly plots to display, you'll have to set the figures to display in your web browser.
Thus the reason for io.renderers.default='browser'

In [1]:
!pip install yfinance
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install plotly

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.io as io
io.renderers.default='browser'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Make_graph Function
Use the following function to plot stock data with the plotly library.
- stock_data: a dataframe with stock data - must contain "Date" and "Close" columns
- revenue_data: a dataframe with revenue data - must contain "Date" and "Close" columns
- stock: name of the stock

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

#### 1. Extract Tesla stock data using the Ticker function.
Then extract the historical data using the .history() method. Set the period to 'max' to obtain the history of Tesla's stock since the beginning.

In [3]:
tesla = yf.Ticker('TSLA') # ticker object holding stock data for TSLA

teslaShareData = tesla.history(period = 'max')
# TSLA historical data. Period = 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 
# 1wk, 1mo, 3mo, max.
#
# Dataframe.index: Index = "Date" column 
# Dataframe.columns: Headers = ["Open", "High", "Low", ...]
print('Tesla Share Data: ')
print(teslaShareData.head())

Tesla Share Data: 
                               Open      High       Low     Close     Volume  \
Date                                                                           
2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667  281494500   
2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667  257806500   
2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000  123282000   
2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   77097000   
2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000  103003500   

                           Dividends  Stock Splits  
Date                                                
2010-06-29 00:00:00-04:00        0.0           0.0  
2010-06-30 00:00:00-04:00        0.0           0.0  
2010-07-01 00:00:00-04:00        0.0           0.0  
2010-07-02 00:00:00-04:00        0.0           0.0  
2010-07-06 00:00:00-04:00        0.0           0.0  


Reset the index of the dataframe and display the first 5 rows.

In [4]:
teslaShareData = teslaShareData.reset_index()
# Resets the index from the "Date" column to a list of [0, 1, 2, ...]
# "Date" column now part of dataframe. 

print('Tesla Share Data (Indices Revised): ')
print(teslaShareData.head())

Tesla Share Data (Indices Revised): 
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


An alternative approach to resetting the indicies:

teslaShareData.reset_index(inplace=True)

print(teslaShareData.head())
    
- inplace=True: actively revises indices of the dataframe.
- inplace=False: creates a copy of the dataframe, which you'll have to assign.

#### 2. Use the request library to download Tesla revenue data. Save the text response in html_data

<div><a href = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'>Link to Tesla revenue data</a></div>

In [5]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'    
response = requests.get(url) # retrieve data from server
html_data = response.text # stores the html code as a string

soup = BeautifulSoup(html_data,"html.parser")

Extract the table "Tesla Quarterly Revenue" and store it in dataframe 'tesla_revenue'. The dataframe should contain columns "Date" and "Revenue".

##### First, determine the index of the table with the title "Tesla Quarterly Revenue"

In [6]:
theaders = soup.find_all('th') # extracts the table headers of all tables in a list
# soup.find_all('th'):
    # [ <th> ... </th>,
    #   <th> ... </th>,
    # ] - there will be 6 tables in the website.

How I know 6?
- View the source code of the site.
- Then hit Ctrl+F --> "<thead>" --> Browser tells me there's 6 instances of this.

In [7]:
index = 0
for i, header in enumerate(theaders):
    # loop through each header, i.e. each element in the list theaders
    if header.text == 'Tesla Quarterly Revenue(Millions of US $)':
        #print('Index: ',i)
        #print(header)
        index = i
        # Assign the index of the element with 'Tesla Quarterly ...' into 'index'

##### Second, extract the data from the table into dataframe tesla_revenue

In [8]:
columns = ['Date','Revenue']
tesla_revenue = pd.DataFrame(columns = columns) # set the columns of data frame

date = []
revenue = []

tablebody = soup.find_all('tbody') # Focus only on the stuff within <tbody> 
tesla_revbody = tablebody[index] # Consider only the <tbody> corresponding to Tesla Quarterly Revenue

# Loop through each row of the table, <tr> ... </tr>
for row in tesla_revbody.find_all('tr'):
    # tesla_revbody.find_all('tr')
    # [ <tr><td>2022-09-30</td><td>$21,454</td></tr>, 
    #   <tr><td>2022-06-30</td><td>$16,934</td></tr>
    #  ... ]
    
    # 1st row: <tr><td>2022-09-30</td><td>$21,454</td></tr>
    
    cell = row.find_all('td')
    # for 1st row, cell = row.find_all('td')
    # [ <td>2022-09-30</td>,
    #   <td>$21,454</td>]
    
    date.append(cell[0].text)
    revenue.append(cell[1].text)

tesla_revenue[columns[0]] = date
tesla_revenue[columns[1]] = revenue

print('Tesla Quarterly Revenue (Millions of US $): ')
print(tesla_revenue)
print(tesla_revenue.loc[tesla_revenue.index[-1],'Date'])
print(tesla_revenue.loc[tesla_revenue.index[-1],'Revenue'])


Tesla Quarterly Revenue (Millions of US $): 
          Date  Revenue
0   2022-09-30  $21,454
1   2022-06-30  $16,934
2   2022-03-31  $18,756
3   2021-12-31  $17,719
4   2021-09-30  $13,757
5   2021-06-30  $11,958
6   2021-03-31  $10,389
7   2020-12-31  $10,744
8   2020-09-30   $8,771
9   2020-06-30   $6,036
10  2020-03-31   $5,985
11  2019-12-31   $7,384
12  2019-09-30   $6,303
13  2019-06-30   $6,350
14  2019-03-31   $4,541
15  2018-12-31   $7,226
16  2018-09-30   $6,824
17  2018-06-30   $4,002
18  2018-03-31   $3,409
19  2017-12-31   $3,288
20  2017-09-30   $2,985
21  2017-06-30   $2,790
22  2017-03-31   $2,696
23  2016-12-31   $2,285
24  2016-09-30   $2,298
25  2016-06-30   $1,270
26  2016-03-31   $1,147
27  2015-12-31   $1,214
28  2015-09-30     $937
29  2015-06-30     $955
30  2015-03-31     $940
31  2014-12-31     $957
32  2014-09-30     $852
33  2014-06-30     $769
34  2014-03-31     $621
35  2013-12-31     $615
36  2013-09-30     $431
37  2013-06-30     $405
38  2013-03-31     

Remove punctuation, and NaN values.

In [9]:
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace('$','').str.replace(',','')    
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]   
print('Tesla Quarterly Revenue (Millions of US $) (Revised): ')
print(tesla_revenue)   

Tesla Quarterly Revenue (Millions of US $) (Revised): 
          Date Revenue
0   2022-09-30   21454
1   2022-06-30   16934
2   2022-03-31   18756
3   2021-12-31   17719
4   2021-09-30   13757
5   2021-06-30   11958
6   2021-03-31   10389
7   2020-12-31   10744
8   2020-09-30    8771
9   2020-06-30    6036
10  2020-03-31    5985
11  2019-12-31    7384
12  2019-09-30    6303
13  2019-06-30    6350
14  2019-03-31    4541
15  2018-12-31    7226
16  2018-09-30    6824
17  2018-06-30    4002
18  2018-03-31    3409
19  2017-12-31    3288
20  2017-09-30    2985
21  2017-06-30    2790
22  2017-03-31    2696
23  2016-12-31    2285
24  2016-09-30    2298
25  2016-06-30    1270
26  2016-03-31    1147
27  2015-12-31    1214
28  2015-09-30     937
29  2015-06-30     955
30  2015-03-31     940
31  2014-12-31     957
32  2014-09-30     852
33  2014-06-30     769
34  2014-03-31     621
35  2013-12-31     615
36  2013-09-30     431
37  2013-06-30     405
38  2013-03-31     562
39  2012-12-31     306
40

#### 3. Extract Gamestop stock data using the Ticker function.

In [10]:
GME = yf.Ticker('GME') # ticker object holding stock data for TSLA

gme_Data = GME.history(period = 'max')
# GME historical data. Period = 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 
# 1wk, 1mo, 3mo, max.

# Reset the index of the dataframe and display the first 5 rows.
gme_Data = gme_Data.reset_index()
# Resets the index from the "Date" column to a list of [0, 1, 2, ...]
# "Date" column now part of dataframe. 

print('GME Share Data (Indices Revised): ')
print(gme_Data.head())

GME Share Data (Indices Revised): 
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


#### 4. Extract Gamestop revenue data using requests. Save the text of the response in the variable html_data
<div><a href='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'>Link to GameStop Revenue Data</a></div>

In [11]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
response = requests.get(url) # retrieve data from server
html_data = response.text # stores the html code as a string

soup = BeautifulSoup(html_data,"html.parser")

Extract the table "Gamestop Quarterly Revenue" and store it in dataframe
'gme_revenue'. The dataframe should contain columns "Date" and "Revenue".
Remove the commas and dollar signs from the "Revenue" column.

In [12]:
# Determine the index of the table with the title "GamepStop Quarterly Revenue"
theaders2 = soup.find_all('th') # extracts the table headers of all tables in a list
# soup.find_all('th'):
    # [ <th> ... </th>,
    #   <th> ... </th>,
    # ] - there will be 6 tables in the website.
    
index = 0
for i, header in enumerate(theaders2):
    # loop through each header, i.e. each element in the list theaders
    if header.text == 'GameStop Quarterly Revenue(Millions of US $)':
        print('Index: ',i)
        print(header)
        index = i
        # Assign the index of the element with 'GameStop Quarterly ...' into 'index'

# Extract the data from the table into dataframe gme_revenue
columns = ['Date','Revenue']
gme_revenue = pd.DataFrame(columns = columns) # set the columns of data frame

# Populate the dataframe
date = []
revenue = []

tablebody = soup.find_all('tbody') # Focus only on the stuff within <tbody> 
gme_revbody = tablebody[index] # Consider only the <tbody> corresponding to GameStop Quarterly Revenue

# Loop through each row of the table, <tr> ... </tr>
for row in gme_revbody.find_all('tr'):
    # gme_revbody.find_all('tr')
    # [ <tr><td>2020-04-30</td><td>$1,021</td></tr>, 
    #   <tr><td>2020-01-31</td><td>$2,194</td></tr>
    #  ... ]
    
    # 1st row: <tr><td>2020-04-30</td><td>$1,021</td></tr>
    
    cell = row.find_all('td')
    # for 1st row, cell = row.find_all('td')
    # [ <td>2020-04-30</td>,
    #   <td>$1,021</td>]
    
    date.append(cell[0].text)
    revenue.append(cell[1].text)

gme_revenue[columns[0]] = date
gme_revenue[columns[1]] = revenue

print('Check if the below line represents the revenue in 2005-01-31 (which is $709 as per the hyperlink)')
print(gme_revenue.loc[gme_revenue.index[-1],'Date'])
print(gme_revenue.loc[gme_revenue.index[-1],'Revenue'])

Index:  1
<th colspan="2" style="text-align:center">GameStop Quarterly Revenue<br/><span style="font-size:14px;">(Millions of US $)</span></th>
Check if the below line represents the revenue in 2005-01-31 (which is $709 as per the hyperlink)
2005-01-31
$709


Remove punctuation, and NaN values.

In [13]:
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace('$','').str.replace(',','')    
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]   
print('GameStop Quarterly Revenue (Millions of US $) (Revised): ')
print(gme_revenue)   

GameStop Quarterly Revenue (Millions of US $) (Revised): 
          Date Revenue
0   2020-04-30    1021
1   2020-01-31    2194
2   2019-10-31    1439
3   2019-07-31    1286
4   2019-04-30    1548
..         ...     ...
57  2006-01-31    1667
58  2005-10-31     534
59  2005-07-31     416
60  2005-04-30     475
61  2005-01-31     709

[62 rows x 2 columns]


#### 5. Use make_graph function to graph the Tesla stock data. Provide a title for the graph. Note that the graph will show data up to June 2021.
Note that the graph will be displayed in a separate browser tab.

In [14]:
make_graph(teslaShareData, tesla_revenue, 'Tesla')

C:\Users\nicco\AppData\Local\Temp\ipykernel_16512\3433895755.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\nicco\AppData\Local\Temp\ipykernel_16512\3433895755.py:7: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



#### 6. Use make_graph function to graph the GameStop stock data. Provide a title for the graph. Note that the graph will show data up to June 2021.
The graph will be displayed in a separate tab within the browser.

In [15]:
make_graph(gme_Data, gme_revenue, 'Gamestop')

C:\Users\nicco\AppData\Local\Temp\ipykernel_16512\3433895755.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\nicco\AppData\Local\Temp\ipykernel_16512\3433895755.py:7: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

